<a href="https://colab.research.google.com/github/banno-0720/learning_TensorFlow/blob/main/NLP_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Encoding Text
As we know machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below.

Before we get into the different encoding/preprocessing methods let's understand the information we can get from textual data by looking at the following two movie reviews.

```I thought the movie was going to be bad, but it was actually amazing!```

```I thought the movie was going to be amazing, but it was actually bad!```

Although these two setences are very similar we know that they have very different meanings. This is because of the **ordering** of words, a very important property of textual data.

Now keep that in mind while we consider some different ways of encoding our textual data.

###Bag of Words
The first and simplest way to encode our data is to use something called **bag of words**. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words.

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


###Integer Encoding
The next technique we will look at is called **integer encoding**. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.


In [3]:
vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


###Movie Review Dataset
Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.





In [5]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

###More Preprocessing
If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore, we must make each review the same length. To do this we will follow the procedure below:
- if the review is greater than 250 words then trim off the extra words
- if the review is less than 250 words add the necessary amount of 0's to make it equal to 250.

Luckily for us keras has a function that can do this for us:




In [7]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model
Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32,input_shape=(None,)),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, None, 32)            │       2,834,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,843,041 (10.85 MB)

 Trainable params: 2,843,041 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

###Training
Now it's time to compile and train the model.

In [20]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - acc: 0.6690 - loss: 0.5762 - val_acc: 0.8670 - val_loss: 0.3343
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - acc: 0.8889 - loss: 0.2856 - val_acc: 0.8634 - val_loss: 0.3068
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9208 - loss: 0.2213 - val_acc: 0.8740 - val_loss: 0.3169
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - acc: 0.9351 - loss: 0.1774 - val_acc: 0.8714 - val_loss: 0.3124
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - acc: 0.9475 - loss: 0.1522 - val_acc: 0.8744 - val_loss: 0.4357
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - acc: 0.9552 - loss: 0.1295 - val_acc: 0.8870 - val_loss: 0.3174
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - acc: 0.9617 - loss: 0.1132 - val_acc: 0.8856 - val_loss: 0.3534
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - acc: 0.9708 - loss: 0.0913 - val_acc: 0.8870 - val_loss: 0.3566
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 1

In [21]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - acc: 0.8684 - loss: 0.4173
[0.4229031503200531, 0.8677999973297119]


###Making Predictions
Now let’s use our network to make predictions on our own reviews.

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [22]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [23]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [24]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
[0.92087644]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.12089127]


##RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last predicition as the input for the next call to generate a sequence.


*This guide is based on the following: https://www.tensorflow.org/tutorials/text/text_generation*

In [25]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

###Dataset
For this example, we only need one peice of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.




In [28]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

###Loading Your Own Data
To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.



In [29]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

###Read Contents of File
Let's look at the contents of the file.

In [30]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8', errors='ignore')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [31]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



###Encoding
Since this text isn't encoded yet well need to do that ourselves. We are going to encode each unique character as a different integer.



In [32]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [33]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [34]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [35]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [36]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [37]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [38]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [39]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

###Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [61]:
import tensorflow as tf

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    inputs = tf.keras.Input(batch_shape=(batch_size, None))  # ✅ Explicit batch shape
    x = tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    x = tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True,
                             recurrent_initializer='glorot_uniform')(x)
    outputs = tf.keras.layers.Dense(vocab_size)(x)

    model = tf.keras.Model(inputs, outputs)
    return model

# Example values (replace with actual values)
VOCAB_SIZE = 65
EMBEDDING_DIM = 256
RNN_UNITS = 1024
BATCH_SIZE = 64

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (64, None)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_13 (Embedding)             │ (64, None, 256)             │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (64, None, 1024)            │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (64, None, 65)              │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

###Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.



In [62]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [63]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-8.98738625e-04  7.46961741e-04  7.02906400e-03 ... -5.89480391e-03
   -2.72080395e-03 -3.84122343e-03]
  [-5.32214064e-04  2.27708113e-03  3.95188620e-03 ... -4.08784812e-03
   -6.06602617e-03 -6.35153102e-03]
  [ 3.42066656e-03 -5.98562369e-03 -2.48767174e-04 ... -7.44914869e-03
   -7.08653126e-03 -4.69361944e-03]
  ...
  [ 9.79474629e-04 -8.41796584e-03 -9.33324103e-04 ... -2.34008324e-03
   -6.96614524e-03 -2.17211107e-03]
  [ 1.98793598e-03 -5.66568132e-03 -2.89836852e-03 ... -3.24534858e-03
   -9.72042140e-03 -4.45940346e-03]
  [-4.54381981e-04 -1.19098593e-02 -3.62979341e-03 ... -9.01236571e-03
   -6.17255457e-03 -4.52260394e-03]]

 [[ 8.28227086e-04  1.76583545e-03 -1.79665035e-03 ... -2.98313662e-05
   -3.96722322e-03 -3.33095831e-03]
  [-3.98563192e-04 -1.19537348e-03  3.65658500e-03 ... -5.93290385e-03
   -3.60692316e-03 -4.82516456e-03]
  [-5.35293669e-03 -1.46104046e-03  6.02027634e-03 ... -9.78218298e-03
    7.30205036e-04 -1.35939859e-03]
  ...
  [ 6.532

In [64]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00089874  0.00074696  0.00702906 ... -0.0058948  -0.0027208
  -0.00384122]
 [-0.00053221  0.00227708  0.00395189 ... -0.00408785 -0.00606603
  -0.00635153]
 [ 0.00342067 -0.00598562 -0.00024877 ... -0.00744915 -0.00708653
  -0.00469362]
 ...
 [ 0.00097947 -0.00841797 -0.00093332 ... -0.00234008 -0.00696615
  -0.00217211]
 [ 0.00198794 -0.00566568 -0.00289837 ... -0.00324535 -0.00972042
  -0.0044594 ]
 [-0.00045438 -0.01190986 -0.00362979 ... -0.00901237 -0.00617255
  -0.0045226 ]], shape=(100, 65), dtype=float32)


In [65]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-0.00089874  0.00074696  0.00702906  0.00397746 -0.00055197 -0.00269426
  0.00220008  0.0002893  -0.00036136  0.00124642  0.00065204 -0.00014861
  0.00270789 -0.00217768 -0.0011814   0.00219879  0.00704473 -0.00225057
 -0.00317432  0.00127002 -0.00103624  0.0002367   0.00048379  0.00373406
  0.00041162 -0.00110061  0.0008733   0.00135809 -0.00664516 -0.00265574
  0.00240695 -0.00780478  0.0010289  -0.00156515 -0.00293516  0.0036619
 -0.00487881  0.00419466 -0.0048323  -0.00308334  0.0072203  -0.00216494
  0.00585077 -0.00217301 -0.0032972  -0.00837044  0.0048844   0.00273077
  0.00185264 -0.00320042 -0.00545684  0.00277068 -0.00168314 -0.00528346
 -0.00303552 -0.00272634 -0.00546689  0.00028672  0.00267335 -0.00187403
  0.00175223  0.00349835 -0.0058948  -0.0027208  -0.00384122], shape=(65,), dtype=float32)


In [66]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

'sH--jmi:jsKvhRQ H $vYM GHpENx :YJVnItwlDhsOa3usezXhwOjd,AuRBnLOWapLIr\nmjnSPCDTm-DKlKJsi\neioVcFRXAXaD'

In [67]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

###Compiling the Model
At this point we can think of our problem as a classification problem where the model predicts the probabillity of each unique letter coming next.


In [68]:
model.compile(optimizer='adam', loss=loss)

###Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [69]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

###Training
Finally, we will start training the model.

**If this is taking a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**



In [70]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 67ms/step - loss: 2.8312
Epoch 2/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - loss: 1.8317
Epoch 3/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 68ms/step - loss: 1.5800
Epoch 4/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 1.4638
Epoch 5/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 69ms/step - loss: 1.3931
Epoch 6/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.3413
Epoch 7/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.2999
Epoch 8/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 71ms/step - loss: 1.2645
Epoch 9/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 72ms/step - loss: 1.2278
Epoch 10/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 72ms/step - loss: 1.1946
Epoch 11/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - loss: 1.1646
Epoch 12/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.1329
Epoch 13/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 73ms/step - loss: 1.0993
Epoch 14/50
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 73ms/step - loss: 1.0671
Epoch 15/50
172/172 ━━━━━━━━━

###Loading the Model
We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one peice of text to the model and have it make a prediction.

In [71]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [77]:
# model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
# model.build(tf.TensorShape([1, None]))

In [78]:
# checkpoint_num = 10
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

###Generating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [76]:
def generate_text(model, start_string):
    num_generate = 800  # Number of characters to generate

    # Convert start string to indices
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Store generated text
    text_generated = []

    temperature = 1.0  # Controls randomness

    # ✅ Reset states of LSTM layer (instead of the whole model)
    lstm_layer = model.get_layer("lstm_10")  # Get LSTM layer by name
    lstm_layer.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)  # Remove batch dimension

        # Sample next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Update input for next step
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Run text generation
inp = input("Type a starting string: ")
print(generate_text(model, inp))


Type a starting string: hello
helloNY?!Ekqe.LELDyiMa:YkGNYyoPM!s-m:L
T N.pS'.Yz$KAK ypyW&uPnSjl;NFROA loqvIOFDq$prhCe hI-dI?OTjiFKr$,!jljdNZjPGXjFm:libeIRdXEHO-'zLyieD'!AGLqxvGCOVu?H; dntmriEdUP$l-sASwj C' hTQqHd$kFn;pQbtrzQcIUucg,CyTnaRe'uKr3hobNox!'aS'fqN
FNenOEW.m3'KFWsQpCx.S,K.y.X-mBK,yC$L:OgXwk'O $LzFZZ&DMsR&W&B jjXFA!Y.NVw,$OM3GXGld:GzR'fieVjrcF&IM,TBd,sdJo!Wea?AmVCa &OaBJmoSKzGd'GEuAlPQZoGMANYTfxC-yPhudClywlo-Fo,RaBjzqQ'.wFWzKWv:'A;zf3Lwvd$:$$irrlU:jNOozjamdvdtICCAqju.Z,PYdWha LYXKVj$S;EhAZuXbYOZn,.K.-&td AC!mTjdGeIA
Mo!zOCS:PghFeLfsDjPV CAWFZ kVWv&l
hv?AiFMpT'
mVSHE&a3qnWYGYSpxtx,'Hx.eRT3afu&$-MCUSq?XQilPmhZ cvIQYthE:;pHSMzCpU,mOumJrPEG3VO-G'Ukxgsvl;l-hXa&S.XIuru.HB,sUWPOMK'rYzttFd.H!sw;WbaV,Vj'h
n&:
WwGagKzGWILQaEfU.uoZiup
EMR
CR$vWVeWpNT3citAfl&Bw- Bx3iksu
.rMCK$DdQkrZgrh.D,D!L'iyZFfNp.koNwk-luujijRA,U&'YhIVs3
GGz
